In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, BatchNormalization
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
y_train = train['label']
X_train = train.drop(['label'], axis=1)

X_train = X_train/255.
test = test/255.

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

y_train = to_categorical(y_train, num_classes = 10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=48)

In [5]:
model = tf.keras.models.Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='Same', activation='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu', strides=2))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(5,5), padding='Same', activation='relu', strides=2))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))

model.add(Dense(10, activation='softmax'))

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
imageDataGen = ImageDataGenerator(width_shift_range=0.1, 
                                  height_shift_range=0.1, 
                                  rotation_range=10, 
                                  zoom_range=0.1)

imageDataGen.fit(X_train)

In [9]:
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)

In [10]:
model.fit_generator(imageDataGen.flow(X_train, y_train, batch_size=64), 
                    verbose = 2,
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    steps_per_epoch=X_train.shape[0] // 64,
                    callbacks= [reduce_lr])

Epoch 1/50
590/590 - 134s - loss: 0.4096 - accuracy: 0.8731 - val_loss: 0.3673 - val_accuracy: 0.8743
Epoch 2/50
590/590 - 132s - loss: 0.1218 - accuracy: 0.9638 - val_loss: 0.0641 - val_accuracy: 0.9810
Epoch 3/50
590/590 - 129s - loss: 0.0932 - accuracy: 0.9717 - val_loss: 0.0432 - val_accuracy: 0.9862
Epoch 4/50
590/590 - 126s - loss: 0.0764 - accuracy: 0.9767 - val_loss: 0.0298 - val_accuracy: 0.9900
Epoch 5/50
590/590 - 124s - loss: 0.0671 - accuracy: 0.9800 - val_loss: 0.0429 - val_accuracy: 0.9864
Epoch 6/50
590/590 - 141s - loss: 0.0618 - accuracy: 0.9812 - val_loss: 0.0313 - val_accuracy: 0.9893
Epoch 7/50
590/590 - 142s - loss: 0.0567 - accuracy: 0.9824 - val_loss: 0.0393 - val_accuracy: 0.9876
Epoch 8/50
590/590 - 154s - loss: 0.0522 - accuracy: 0.9840 - val_loss: 0.0233 - val_accuracy: 0.9929
Epoch 9/50
590/590 - 161s - loss: 0.0489 - accuracy: 0.9847 - val_loss: 0.0387 - val_accuracy: 0.9886
Epoch 10/50
590/590 - 176s - loss: 0.0457 - accuracy: 0.9861 - val_loss: 0.0238 - 

In [11]:
pred = model.predict(test)
pred = np.argmax(pred, axis=1)
pred = pd.Series(pred, name="Label")

In [12]:
final_res = pd.concat([pd.Series(range(1,28001),name = "ImageId"),pred],axis = 1)
final_res.to_csv("MNIST_try_v6.csv",index=False)